## Demo 5.1: Vcluster with DataStore Persistence (Default Behaviour)

### Create vcluster

In [1]:
! vcluster create vc1 -n host-namespace --expose

[info]   Creating namespace host-namespace
[info]   execute command: helm upgrade vc1 vcluster --repo https://charts.loft.sh --version 0.8.1 --kubeconfig /tmp/3137757920 --namespace host-namespace --install --repository-config='' --values /tmp/3030171424
[done] √ Successfully created virtual cluster vc1 in namespace host-namespace. 
- Use 'vcluster connect vc1 --namespace host-namespace' to access the virtual cluster
- Use `vcluster connect vc1 --namespace host-namespace -- kubectl get ns` to run a command directly within the vcluster


In [8]:
! kubectl get all -n host-namespace

NAME                                               READY   STATUS    RESTARTS   AGE
pod/coredns-5df468b6b7-fjgd5-x-kube-system-x-vc1   1/1     Running   0          7s
pod/vc1-0                                          2/2     Running   0          21s

NAME                                        TYPE           CLUSTER-IP      EXTERNAL-IP      PORT(S)                  AGE
service/kube-dns-x-kube-system-x-vc1        ClusterIP      10.96.28.21     <none>           53/UDP,53/TCP,9153/TCP   7s
service/vc1                                 LoadBalancer   10.96.187.122   172.18.255.201   443:30642/TCP            21s
service/vc1-headless                        ClusterIP      None            <none>           443/TCP                  21s
service/vc1-node-kcd-vcluster-demo-worker   ClusterIP      10.96.228.239   <none>           10250/TCP                7s

NAME                   READY   AGE
statefulset.apps/vc1   1/1     21s


##### Check that persistence is enabled

######  By verifying the helm values

In [9]:
! echo "storage.persistence:" $(helm get values vc1 -n host-namespace -a | grep -v COMPUTED_VALUES | yq '.storage.persistence')

storage.persistence: true


######  By verifying vc1-0 pod's data volume

In [10]:
! kubectl get pod vc1-0 --namespace host-namespace \
  -o jsonpath="{.spec.volumes[?(@.name == 'data')]}" | jq

{
  "name": "data",
  "persistentVolumeClaim": {
    "claimName": "data-vc1-0"
  }
}


##### Accessing Vcluster

In [11]:
! vcluster connect vc1 --namespace host-namespace

[info]   Using vcluster vc1 load balancer endpoint: 172.18.255.201
[info]   Use `vcluster connect vc1 -n host-namespace -- kubectl get ns` to execute a command directly within this terminal
[done] √ Virtual cluster kube config written to: ./kubeconfig.yaml. You can access the cluster via `kubectl --kubeconfig ./kubeconfig.yaml get namespaces`


##### Create a workload inside the vcluster

In [12]:
! kubectl create ns ns1 --kubeconfig ./kubeconfig.yaml
! kubectl create deployment sample -n ns1 --image=nginx --replicas 2 --kubeconfig ./kubeconfig.yaml

namespace/ns1 created
deployment.apps/sample created


In [15]:
! kubectl get pods -n ns1 --kubeconfig ./kubeconfig.yaml

NAME                      READY   STATUS    RESTARTS   AGE
sample-7f75ccc884-8wd7h   1/1     Running   0          6s
sample-7f75ccc884-zmldt   1/1     Running   0          6s


##### Simulate a vcluster failure by deleting vcluster pod in host cluster

In [16]:
 ! kubectl get pods -n host-namespace

NAME                                           READY   STATUS    RESTARTS   AGE
coredns-5df468b6b7-fjgd5-x-kube-system-x-vc1   1/1     Running   0          115s
sample-7f75ccc884-8wd7h-x-ns1-x-vc1            1/1     Running   0          37s
sample-7f75ccc884-zmldt-x-ns1-x-vc1            1/1     Running   0          37s
vc1-0                                          2/2     Running   0          2m9s


In [17]:
 ! kubectl delete pod vc1-0 -n host-namespace

pod "vc1-0" deleted


In [18]:
! kubectl get pods -n host-namespace

NAME                                           READY   STATUS    RESTARTS   AGE
coredns-5df468b6b7-fjgd5-x-kube-system-x-vc1   1/1     Running   0          2m22s
sample-7f75ccc884-8wd7h-x-ns1-x-vc1            1/1     Running   0          64s
sample-7f75ccc884-zmldt-x-ns1-x-vc1            1/1     Running   0          64s
vc1-0                                          2/2     Running   0          10s


In [19]:
! kubectl get pods -n ns1 --kubeconfig ./kubeconfig.yaml

NAME                      READY   STATUS    RESTARTS   AGE
sample-7f75ccc884-8wd7h   1/1     Running   0          88s
sample-7f75ccc884-zmldt   1/1     Running   0          88s


### Cleanup

In [20]:
! vcluster delete vc1 -n host-namespace
! kubectl delete ns host-namespace

[info]   Delete helm chart with helm delete vc1 --namespace host-namespace --kubeconfig /tmp/573939516 --repository-config=''
[done] √ Successfully deleted virtual cluster vc1 in namespace host-namespace
[done] √ Successfully deleted virtual cluster pvc data-vc1-0 in namespace host-namespace
namespace "host-namespace" deleted


In [21]:
! kubectl get ns

NAME                 STATUS   AGE
default              Active   31m
kube-node-lease      Active   31m
kube-public          Active   31m
kube-system          Active   31m
local-path-storage   Active   31m
metallb-system       Active   30m
